In [1]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v2.4.5 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [2]:
display(dbutils.fs.ls('/mnt/datasets/brazilian-ecommerce/'))

path,name,size
dbfs:/mnt/datasets/brazilian-ecommerce/mkt_datasets_arch.png,mkt_datasets_arch.png,182541
dbfs:/mnt/datasets/brazilian-ecommerce/olist_closed_deals_dataset.csv,olist_closed_deals_dataset.csv,171352
dbfs:/mnt/datasets/brazilian-ecommerce/olist_customers_dataset.csv,olist_customers_dataset.csv,9033957
dbfs:/mnt/datasets/brazilian-ecommerce/olist_ecommerce_datasets_arch.png,olist_ecommerce_datasets_arch.png,209677
dbfs:/mnt/datasets/brazilian-ecommerce/olist_geolocation_dataset.csv,olist_geolocation_dataset.csv,61273883
dbfs:/mnt/datasets/brazilian-ecommerce/olist_marketing_qualified_leads_dataset.csv,olist_marketing_qualified_leads_dataset.csv,703836
dbfs:/mnt/datasets/brazilian-ecommerce/olist_order_items_dataset.csv,olist_order_items_dataset.csv,15438671
dbfs:/mnt/datasets/brazilian-ecommerce/olist_order_payments_dataset.csv,olist_order_payments_dataset.csv,5777138
dbfs:/mnt/datasets/brazilian-ecommerce/olist_order_reviews_dataset.csv,olist_order_reviews_dataset.csv,14409007
dbfs:/mnt/datasets/brazilian-ecommerce/olist_orders_dataset.csv,olist_orders_dataset.csv,17654914


In [3]:
closed_deals = spark.read.csv('/mnt/datasets/brazilian-ecommerce/olist_closed_deals_dataset.csv', header = True, sep = ',', inferSchema=True)
customers = spark.read.csv('/mnt/datasets/brazilian-ecommerce/olist_customers_dataset.csv', header = True, sep = ',', inferSchema=True)
geolocation = spark.read.csv('/mnt/datasets/brazilian-ecommerce/olist_geolocation_dataset.csv', header = True, sep = ',', inferSchema=True)
marketing = spark.read.csv('/mnt/datasets/brazilian-ecommerce/olist_marketing_qualified_leads_dataset.csv', header = True, sep = ',', inferSchema=True)
order_items = spark.read.csv('/mnt/datasets/brazilian-ecommerce/olist_order_items_dataset.csv', header = True, sep = ',', inferSchema=True)
order_payments = spark.read.csv('/mnt/datasets/brazilian-ecommerce/olist_order_payments_dataset.csv', header = True, sep = ',', inferSchema=True)
order_review = spark.read.csv('/mnt/datasets/brazilian-ecommerce/olist_order_reviews_dataset.csv', header = True, sep = ',', inferSchema=True)
orders = spark.read.csv('/mnt/datasets/brazilian-ecommerce/olist_orders_dataset.csv', header = True, sep = ',', inferSchema=True)
products = spark.read.csv('/mnt/datasets/brazilian-ecommerce/olist_products_dataset.csv', header = True, sep = ',', inferSchema=True)
sellers = spark.read.csv('/mnt/datasets/brazilian-ecommerce/olist_sellers_dataset.csv', header = True, sep = ',', inferSchema=True)
product_category = spark.read.csv('/mnt/datasets/brazilian-ecommerce/product_category_name_translation.csv', header = True, sep = ',', inferSchema=True)

In [4]:
closed_deals.dtypes

Out[4]: [('mql_id', 'string'),
 ('seller_id', 'string'),
 ('sdr_id', 'string'),
 ('sr_id', 'string'),
 ('won_date', 'timestamp'),
 ('business_segment', 'string'),
 ('lead_type', 'string'),
 ('lead_behaviour_profile', 'string'),
 ('has_company', 'boolean'),
 ('has_gtin', 'boolean'),
 ('average_stock', 'string'),
 ('business_type', 'string'),
 ('declared_product_catalog_size', 'double'),
 ('declared_monthly_revenue', 'double')]

In [5]:
display(closed_deals.describe())

summary,mql_id,seller_id,sdr_id,sr_id,business_segment,lead_type,lead_behaviour_profile,average_stock,business_type,declared_product_catalog_size,declared_monthly_revenue
count,842,842,842,842,841,836,665,66,832,69,842
mean,null,null,null,null,null,null,null,null,null,233.02898550724638,73377.67933491686
stddev,null,null,null,null,null,null,null,null,null,352.38055844642525,1744799.1768914354
min,000dd3543ac84d906eae52e7c779bb2a,00065220becb8785e2cf78355eb9bf68,068066e24f0c643eb1d089c7dd20cd73,060c0a26f19f4d66b42e0d8796688490,air_conditioning,industry,cat,1-5,manufacturer,1.0,0.0
max,fff8db9478d2fd72df65a67ee6b62f67,ffc470761de7d0232558ba5e786e57b7,fdb16d3cbbeb5798f2f66c4096be026d,fbf4aef3f6915dc0c3c97d6812522f6a,watches,other,wolf,unknown,reseller,2000.0,5.0E7


In [6]:
from pyspark.sql.functions import isnull, when, count, col

aux = []
for c in closed_deals.columns:
  aux.append(count(when(isnull(c), c)).alias(c))

display(closed_deals.select(aux))

mql_id,seller_id,sdr_id,sr_id,won_date,business_segment,lead_type,lead_behaviour_profile,has_company,has_gtin,average_stock,business_type,declared_product_catalog_size,declared_monthly_revenue
0,0,0,0,0,1,6,177,779,778,776,10,773,0


In [7]:
closed_deals = closed_deals.fillna('NA', subset=['business_segment'])
closed_deals = closed_deals.fillna('NA', subset=['lead_type'])
closed_deals = closed_deals.fillna('NA', subset=['lead_behaviour_profile'])
closed_deals = closed_deals.fillna('NA', subset=['has_company'])
closed_deals = closed_deals.fillna('NA', subset=['has_gtin'])
closed_deals = closed_deals.fillna('NA', subset=['average_stock'])
closed_deals = closed_deals.fillna('NA', subset=['business_type'])

In [8]:
closed_deals = closed_deals.dropna()
closed_deals.count()

Out[8]: 44

In [9]:
tamanho_declarado = closed_deals[['declared_product_catalog_size']].agg({'declared_product_catalog_size': 'mean'}).collect()[0][0]
closed_deals = closed_deals.fillna(tamanho_declarado, subset=['declared_product_catalog_size'])

In [10]:
from pyspark.sql.functions import isnull, when, count, col

aux = []
for c in closed_deals.columns:
  aux.append(count(when(isnull(c), c)).alias(c))

display(closed_deals.select(aux))

mql_id,seller_id,sdr_id,sr_id,won_date,business_segment,lead_type,lead_behaviour_profile,has_company,has_gtin,average_stock,business_type,declared_product_catalog_size,declared_monthly_revenue
0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:
display(closed_deals.describe())

summary,mql_id,seller_id,sdr_id,sr_id,business_segment,lead_type,lead_behaviour_profile,average_stock,business_type,declared_product_catalog_size,declared_monthly_revenue
count,44,44,44,44,44,44,44,44,44,44,44
mean,null,null,null,null,null,null,null,null,null,205.0,228386.36363636365
stddev,null,null,null,null,null,null,null,null,null,264.3862572511208,1201323.0345323847
min,010ed37e44e2fdc175b4c5c6c930805a,0d7d5bca59d45d750fb7913b974e9d08,068066e24f0c643eb1d089c7dd20cd73,068066e24f0c643eb1d089c7dd20cd73,audio_video_electronics,industry,cat,1-5,manufacturer,1.0,0.0
max,fe7fdf179234f77c4d0ab2ea82a82c52,f50d9d5fa45cba8c1a6b643987810c34,f42a2bd194f7802ab052a815c8de65b7,de63de0d10a6012430098db33c679b0b,watches,other,wolf,unknown,reseller,1200.0,8000000.0


In [12]:
closed_deals.select('mql_id').distinct().count()

Out[12]: 44

In [13]:
closed_deals.select('mql_id').count()

Out[13]: 44

In [14]:
closed_deals.groupby('mql_id').count().show()

+--------------------+-----+
 mql_id|count|
+--------------------+-----+
0dabd1a1be08da96c...| 1|
cb0f32260247ecac6...| 1|
dada30935bcfc75c6...| 1|
3d2c7bb8be9ea80c1...| 1|
b39ac02ff5021fed1...| 1|
5527123f5a35a024c...| 1|
5ee85f64aafcce6b2...| 1|
54007efbb2ae51f02...| 1|
963637be23e343ea9...| 1|
d768d7e09a6f9ff41...| 1|
8cfebc3e119ed2fe0...| 1|
9b6e0fb52eba76006...| 1|
7e4bb8c8f3bdcbece...| 1|
6e668e75a724c9638...| 1|
33ce1e734d9d50629...| 1|
4fbe9e8a0ace3a44f...| 1|
9c2bfd97ce00ce955...| 1|
3223fed48f30bbdc0...| 1|
7720c6892d447539b...| 1|
e4dcab5f7c33b62eb...| 1|
+--------------------+-----+
only showing top 20 rows

In [15]:
closed_deals.select('seller_id').distinct().count()

Out[15]: 44

In [16]:
closed_deals.select('seller_id').count()

Out[16]: 44

In [17]:
closed_deals.groupby('seller_id').count().show()

+--------------------+-----+
 seller_id|count|
+--------------------+-----+
34fe2ce0770697c1b...| 1|
710b15ab273348b3d...| 1|
f233b575e585413f1...| 1|
7d91f9d655df761fd...| 1|
9485222b0bad29d3b...| 1|
b15dd2f0fabf60d00...| 1|
9169e322ad3419286...| 1|
0d7d5bca59d45d750...| 1|
366b6b05f39997f10...| 1|
7e165d0fd266781db...| 1|
53be10ff134691e94...| 1|
e00c0a3af0718a2ec...| 1|
7c7d0dee362960b1d...| 1|
843c73eda46e28b27...| 1|
b30aab8e0bdb2b387...| 1|
8c6d188ef073e2898...| 1|
90c523952c5622d26...| 1|
3786da2e9afd5d556...| 1|
9966324e28b7fa381...| 1|
40105cb0aec41563d...| 1|
+--------------------+-----+
only showing top 20 rows

In [18]:
closed_deals.select('sdr_id').distinct().count()

Out[18]: 19

In [19]:
closed_deals.select('sdr_id').count()

Out[19]: 44

In [20]:
closed_deals.groupby('sdr_id').count().show()

+--------------------+-----+
 sdr_id|count|
+--------------------+-----+
4b339f9567d060bce...| 2|
084b6fbb10729ed4d...| 1|
09285259593c61296...| 1|
b1b3ec93fc27a19fd...| 1|
0a0fb2b07d841f84f...| 1|
6aa3b86a83d784b05...| 1|
e7dff61b78bebffa7...| 7|
f42a2bd194f7802ab...| 7|
de63de0d10a601243...| 1|
2b63542749aa9caf1...| 2|
e4a6222cdb5b34375...| 1|
33dd30594303eb0eb...| 3|
068066e24f0c643eb...| 2|
282f16e20d50cb18d...| 1|
9749123c950bf8363...| 1|
b90f87164b5f8c2cf...| 1|
45749fb708130f78d...| 5|
370c9f455f93a9a96...| 4|
b34f6eba10f46bf9a...| 2|
+--------------------+-----+

In [21]:
closed_deals.select('sr_id').distinct().count()

Out[21]: 16

In [22]:
closed_deals.select('sr_id').count()

Out[22]: 44

In [23]:
closed_deals.groupby('sr_id').count().show()

+--------------------+-----+
 sr_id|count|
+--------------------+-----+
9d12ef1a7eca3ec58...| 3|
85fc447d336637ba1...| 3|
4b339f9567d060bce...| 3|
34d40cdaf94010a1d...| 4|
0a0fb2b07d841f84f...| 1|
6aa3b86a83d784b05...| 1|
56bf83c4bb35763a5...| 1|
495d4e95a8cf8bbf8...| 1|
de63de0d10a601243...| 5|
068066e24f0c643eb...| 6|
9749123c950bf8363...| 7|
a8387c01a09e99ce0...| 1|
2695de1affa775008...| 1|
b90f87164b5f8c2cf...| 1|
d3d1e91a157ea7f90...| 5|
4ef15afb4b2723d8f...| 1|
+--------------------+-----+

In [24]:
closed_deals.select('won_date').distinct().count()

Out[24]: 44

In [25]:
closed_deals.select('won_date').count()

Out[25]: 44

In [26]:
closed_deals.groupby('won_date').count().show()

+-------------------+-----+
 won_date|count|
+-------------------+-----+
2018-08-21 12:30:34| 1|
2018-11-14 18:04:19| 1|
2018-09-10 18:10:00| 1|
2018-11-12 19:17:25| 1|
2018-10-29 16:54:59| 1|
2018-08-14 13:56:57| 1|
2018-08-17 14:41:00| 1|
2018-08-20 13:59:51| 1|
2018-08-23 21:56:01| 1|
2018-11-06 19:41:39| 1|
2018-09-13 03:00:00| 1|
2018-10-02 21:25:39| 1|
2018-08-24 11:54:08| 1|
2018-10-10 01:15:56| 1|
2018-10-30 18:21:05| 1|
2018-09-05 14:32:37| 1|
2018-09-26 11:48:53| 1|
2018-08-22 13:14:44| 1|
2018-08-20 14:41:13| 1|
2018-09-21 18:18:15| 1|
+-------------------+-----+
only showing top 20 rows

In [27]:
closed_deals.select('business_segment').distinct().count()

Out[27]: 17

In [28]:
closed_deals.select('business_segment').count()

Out[28]: 44

In [29]:
closed_deals.groupby('business_segment').count().show()

+--------------------+-----+
 business_segment|count|
+--------------------+-----+
 bed_bath_table| 2|
 computers| 2|
 home_decor| 6|
 other| 2|
 food_supplement| 1|
 party| 1|
 household_utilities| 3|
 health_beauty| 5|
 watches| 1|
audio_video_elect...| 3|
 fashion_accessories| 2|
construction_tool...| 6|
 food_drink| 1|
 toys| 3|
 phone_mobile| 1|
 perfume| 2|
 car_accessories| 3|
+--------------------+-----+

In [30]:
closed_deals.select('lead_type').distinct().count()

Out[30]: 7

In [31]:
closed_deals.select('lead_type').count()

Out[31]: 44

In [32]:
closed_deals.groupby('lead_type').count().show()

#É possivel perceber que o tipo de lead "online_medium" é o que mais possui registros, seguido do "online_big".

+---------------+-----+
 lead_type|count|
+---------------+-----+
 industry| 4|
online_beginner| 3|
 offline| 4|
 online_big| 6|
 online_small| 5|
 other| 2|
 online_medium| 20|
+---------------+-----+

In [33]:
closed_deals.select('lead_behaviour_profile').distinct().count()

Out[41]: 8

In [34]:
closed_deals.select('lead_behaviour_profile').count()

Out[42]: 44

In [35]:
closed_deals.groupby('lead_behaviour_profile').count().show()

#O perfil "cat" aparece 407 vezes, o que representa 48,3% da base. Nessa base há 177 nulos. 

+----------------------+-----+
lead_behaviour_profile|count|
+----------------------+-----+
 eagle| 3|
 shark, wolf| 1|
 wolf| 11|
 cat| 16|
 shark| 5|
 eagle, cat| 2|
 cat, wolf| 5|
 eagle, wolf| 1|
+----------------------+-----+

In [36]:
closed_deals.select('has_company').distinct().count()

Out[44]: 2

In [37]:
closed_deals.groupby('has_company').count().show()

#Há muito nulo nessa coluna!

+-----------+-----+
has_company|count|
+-----------+-----+
 true| 42|
 false| 2|
+-----------+-----+

In [38]:
closed_deals.select('has_company').distinct().count()

Out[46]: 2

In [39]:
closed_deals.groupby('has_gtin').count().show()

+--------+-----+
has_gtin|count|
+--------+-----+
 true| 37|
 false| 7|
+--------+-----+

In [40]:
closed_deals.select('average_stock').distinct().count()

Out[48]: 7

In [41]:
closed_deals.groupby('average_stock').count().show()

+-------------+-----+
average_stock|count|
+-------------+-----+
 200+| 2|
 NA| 6|
 5-20| 14|
 unknown| 4|
 50-200| 8|
 1-5| 6|
 20-50| 4|
+-------------+-----+

In [42]:
closed_deals.select('business_type').distinct().count()

Out[50]: 3

In [43]:
closed_deals.groupby('business_type').count().show()

#É possível observar que mais da metada dos tipos de negócios são de "reseller", seguido de "manufacturer".

+-------------+-----+
business_type|count|
+-------------+-----+
 reseller| 33|
 other| 2|
 manufacturer| 9|
+-------------+-----+

In [44]:
closed_deals.select('declared_product_catalog_size').distinct().count()

Out[52]: 25

In [45]:
closed_deals.groupby('declared_product_catalog_size').count().show()

#Há muito nulo!

+-----------------------------+-----+
declared_product_catalog_size|count|
+-----------------------------+-----+
 305.0| 1|
 70.0| 2|
 75.0| 1|
 47.0| 1|
 120.0| 1|
 1.0| 1|
 300.0| 4|
 700.0| 1|
 85.0| 1|
 50.0| 5|
 45.0| 1|
 400.0| 3|
 200.0| 2|
 10.0| 2|
 40.0| 2|
 30.0| 1|
 1200.0| 1|
 100.0| 5|
 20.0| 2|
 5.0| 1|
+-----------------------------+-----+
only showing top 20 rows

In [46]:
closed_deals.select('declared_monthly_revenue').distinct().count()

Out[54]: 21

In [47]:
display(closed_deals.describe('declared_monthly_revenue'))

summary,declared_monthly_revenue
count,44
mean,228386.36363636365
stddev,1201323.0345323847
min,0.0
max,8000000.0


In [48]:
closed_deals.groupby('declared_monthly_revenue').count().show()

+------------------------+-----+
declared_monthly_revenue|count|
+------------------------+-----+
 300000.0| 1|
 0.0| 15|
 100000.0| 1|
 25000.0| 3|
 130000.0| 1|
 210000.0| 1|
 8000000.0| 1|
 150000.0| 1|
 10000.0| 2|
 250000.0| 2|
 20000.0| 1|
 6000.0| 1|
 4000.0| 1|
 60000.0| 1|
 120000.0| 2|
 15000.0| 2|
 30000.0| 3|
 50000.0| 2|
 8000.0| 1|
 1000.0| 1|
+------------------------+-----+
only showing top 20 rows

In [49]:
customers.dtypes

Out[57]: [('customer_id', 'string'),
 ('customer_unique_id', 'string'),
 ('customer_zip_code_prefix', 'int'),
 ('customer_city', 'string'),
 ('customer_state', 'string')]

In [50]:
display(customers.describe())

summary,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
count,99441,99441,99441,99441,99441
mean,null,null,35137.47458291851,null,null
stddev,null,null,29797.93899620613,null,null
min,00012a2ce6f8dcda20d059ce98491703,0000366f3b9a7992bf8c76cfdf3221e2,1003,abadia dos dourados,AC
max,ffffe8b65bbe3087b653a978c870db99,ffffd2657e2aad2907e67c3e9daecbeb,99990,zortea,TO


In [51]:
from pyspark.sql.functions import isnull, when, count, col

aux = []
for c in customers.columns:
  aux.append(count(when(isnull(c), c)).alias(c))

display(customers.select(aux))

customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,0,0,0,0


In [52]:
customers.select('customer_id').distinct().count()

Out[60]: 99441

In [53]:
customers.select('customer_id').count()

Out[61]: 99441

In [54]:
customers.groupby('customer_id').count().show()

+--------------------+-----+
 customer_id|count|
+--------------------+-----+
e3c7e245a96d7fa33...| 1|
a56b03f5e6015f1a5...| 1|
d0615859a639a94c1...| 1|
c0fe0fbc24994167d...| 1|
5b5f4957a69d537a2...| 1|
41b200d1ce8675f15...| 1|
456c1e01c8ed3b83a...| 1|
8baeca32aac79a831...| 1|
a8004a3d658be3bb2...| 1|
860ac166573be76ff...| 1|
4f8f4d1895013d138...| 1|
e2431d4ec21d4a66b...| 1|
b59bbdb15afd9b563...| 1|
feedd5ec61cfc3b21...| 1|
441a64d207dab5faa...| 1|
5bdebe0bf44ced6aa...| 1|
d6c6231fe530ce9dc...| 1|
8d53cffa847c8c98f...| 1|
982bd9951046d28f8...| 1|
72ecfc69f7d90359d...| 1|
+--------------------+-----+
only showing top 20 rows

In [55]:
customers.select('customer_unique_id').distinct().count()

Out[63]: 96096

In [56]:
customers.select('customer_unique_id').count()

Out[64]: 99441

In [57]:
customers.groupby('customer_unique_id').count().show()

+--------------------+-----+
 customer_unique_id|count|
+--------------------+-----+
861eff4711a542e4b...| 1|
e607ede0e63436308...| 1|
28da048f094c0c9cb...| 1|
212c759d8c4f2d4d9...| 1|
f96176e892232662d...| 1|
000c8bdb58a29e711...| 1|
9ccbb5f759db041b2...| 1|
4b384b778ebc0449d...| 2|
4df43d4c7d3a093a5...| 1|
4be1583defacacfea...| 1|
c85df1c6ef6f7bb60...| 2|
1a3d1e8b28c0ee27c...| 1|
2a6ef69674d6f2800...| 1|
2d6d1699603a346a2...| 1|
5b71ca52f13f79c2b...| 2|
9a176d6883a469743...| 1|
dfe634d4b8e067879...| 1|
5f439c10be261b762...| 1|
cbe71e4cb0b82b97b...| 2|
782d02110160b9efd...| 1|
+--------------------+-----+
only showing top 20 rows

In [58]:
customers.select('customer_zip_code_prefix').distinct().count()

Out[66]: 14994

In [59]:
customers.select('customer_zip_code_prefix').count()

Out[67]: 99441

In [60]:
customers.groupby('customer_zip_code_prefix').count().show()

+------------------------+-----+
customer_zip_code_prefix|count|
+------------------------+-----+
 20735| 37|
 18051| 22|
 13840| 12|
 12940| 21|
 26087| 15|
 29285| 17|
 70355| 3|
 74820| 5|
 31035| 30|
 7240| 16|
 6397| 4|
 4935| 3|
 18800| 19|
 95080| 8|
 2122| 7|
 4101| 5|
 62680| 5|
 2366| 4|
 89874| 10|
 66010| 2|
+------------------------+-----+
only showing top 20 rows

In [61]:
customers.select('customer_city').distinct().count()

Out[69]: 4119

In [62]:
customers.groupby('customer_city').count().show()

+--------------------+-----+
 customer_city|count|
+--------------------+-----+
 camacari| 61|
 arapiraca| 29|
 itaberaba| 10|
 igrejinha| 17|
 aguas de sao pedro| 3|
 vermelho| 1|
 pote| 3|
jijoca de jericoa...| 4|
 barracao| 3|
 iepe| 5|
divino das laranj...| 1|
divino de sao lou...| 1|
 bacaxa| 1|
 redencao da serra| 1|
 astolfo dutra| 11|
 itanhaem| 80|
 brusque| 60|
 boa vista| 45|
 cachoeira paulista| 25|
 guaranta| 5|
+--------------------+-----+
only showing top 20 rows

In [63]:
consumidor_cidade = customers.groupby('customer_city').count()
display(consumidor_cidade.orderBy('count', ascending=False))

customer_city,count
sao paulo,15540
rio de janeiro,6882
belo horizonte,2773
brasilia,2131
curitiba,1521
campinas,1444
porto alegre,1379
salvador,1245
guarulhos,1189
sao bernardo do campo,938


In [64]:
customers.select('customer_state').distinct().count()

Out[72]: 27

In [65]:
customers.groupby('customer_state').count().show()

+--------------+-----+
customer_state|count|
+--------------+-----+
 SC| 3637|
 RO| 253|
 PI| 495|
 AM| 148|
 RR| 46|
 GO| 2020|
 TO| 280|
 MT| 907|
 SP|41746|
 ES| 2033|
 PB| 536|
 RS| 5466|
 MS| 715|
 AL| 413|
 MG|11635|
 PA| 975|
 BA| 3380|
 SE| 350|
 PE| 1652|
 CE| 1336|
+--------------+-----+
only showing top 20 rows

In [66]:
consumidores_estados = customers.groupby('customer_state').count()
display(consumidores_estados.orderBy('count', ascending=False))

customer_state,count
SP,41746
RJ,12852
MG,11635
RS,5466
PR,5045
SC,3637
BA,3380
DF,2140
ES,2033
GO,2020


In [67]:
marketing.dtypes

Out[75]: [('mql_id', 'string'),
 ('first_contact_date', 'timestamp'),
 ('landing_page_id', 'string'),
 ('origin', 'string')]

In [68]:
display(marketing.describe())

summary,mql_id,landing_page_id,origin
count,8000,8000,7940
mean,Infinity,null,null
stddev,NaN,null,null
min,0002ac0d783338cfeab0b2bdbd872cda,007f9098284a86ee80ddeb25d53e0af8,direct_traffic
max,fffffe98d0963d27015c198262d97221,ffc62d6945fd1423539ec6d96326c37c,unknown


In [69]:
from pyspark.sql.functions import isnull, when, count, col

aux = []
for c in marketing.columns:
  aux.append(count(when(isnull(c), c)).alias(c))

display(marketing.select(aux))

mql_id,first_contact_date,landing_page_id,origin
0,0,0,60


In [70]:
marketing = marketing.fillna('NA', subset=['origin'])

In [71]:
marketing = marketing.dropna()
marketing.count()

Out[79]: 8000

In [72]:
from pyspark.sql.functions import isnull, when, count, col

aux = []
for c in marketing.columns:
  aux.append(count(when(isnull(c), c)).alias(c))

display(marketing.select(aux))

mql_id,first_contact_date,landing_page_id,origin
0,0,0,0


In [73]:
display(marketing.describe())

summary,mql_id,landing_page_id,origin
count,8000,8000,8000
mean,Infinity,null,null
stddev,NaN,null,null
min,0002ac0d783338cfeab0b2bdbd872cda,007f9098284a86ee80ddeb25d53e0af8,NA
max,fffffe98d0963d27015c198262d97221,ffc62d6945fd1423539ec6d96326c37c,unknown


In [74]:
marketing.select('mql_id').distinct().count()

Out[82]: 8000

In [75]:
marketing.select('mql_id').count()

Out[83]: 8000

In [76]:
marketing.groupby('mql_id').count().show()

+--------------------+-----+
 mql_id|count|
+--------------------+-----+
e3154fc901362537c...| 1|
c1ca185ea0fad2ac9...| 1|
c52caddded0b8beec...| 1|
ab243ac315c8f9c55...| 1|
5c4ca203172b06327...| 1|
b82d949cf4a38f15b...| 1|
906fec3cca7ccc130...| 1|
3a2cdaaf4d1564030...| 1|
be9ca9311414a5682...| 1|
f5abe18064d57c2e5...| 1|
e5771214664db8f49...| 1|
f2456637128949750...| 1|
a463cac2327534f6f...| 1|
7cd7cb3db3154d9b1...| 1|
95d40515d78b92d75...| 1|
fa959d174416af158...| 1|
8f8161d3a6bdf6a3b...| 1|
fefe447107ab48d50...| 1|
cc033bc7d61c82cbe...| 1|
d2e8ac40dc18011d5...| 1|
+--------------------+-----+
only showing top 20 rows

In [77]:
marketing.select('first_contact_date').distinct().count()

Out[85]: 336

In [78]:
marketing.select('first_contact_date').count()

Out[86]: 8000

In [79]:
marketing.groupby('first_contact_date').count().show()

+-------------------+-----+
 first_contact_date|count|
+-------------------+-----+
2018-05-15 00:00:00| 76|
2018-03-21 00:00:00| 55|
2018-01-31 00:00:00| 49|
2017-08-20 00:00:00| 4|
2018-02-13 00:00:00| 19|
2018-03-25 00:00:00| 13|
2018-05-11 00:00:00| 26|
2017-08-02 00:00:00| 8|
2017-07-17 00:00:00| 9|
2017-11-18 00:00:00| 6|
2017-12-25 00:00:00| 11|
2017-11-13 00:00:00| 21|
2018-04-11 00:00:00| 50|
2017-07-30 00:00:00| 1|
2018-01-02 00:00:00| 34|
2018-02-27 00:00:00| 50|
2017-11-11 00:00:00| 8|
2018-05-25 00:00:00| 50|
2018-05-28 00:00:00| 57|
2017-11-07 00:00:00| 18|
+-------------------+-----+
only showing top 20 rows

In [80]:
primeiro_contato = marketing.groupby('first_contact_date').count()
display(primeiro_contato.orderBy('count', ascending=False))

first_contact_date,count
2018-05-02T00:00:00.000+0000,93
2018-05-15T00:00:00.000+0000,76
2018-01-22T00:00:00.000+0000,75
2018-02-06T00:00:00.000+0000,71
2018-03-26T00:00:00.000+0000,70
2018-03-13T00:00:00.000+0000,69
2018-04-02T00:00:00.000+0000,69
2018-04-18T00:00:00.000+0000,67
2018-04-25T00:00:00.000+0000,66
2018-05-08T00:00:00.000+0000,66


In [81]:
marketing.select('landing_page_id').distinct().count()

Out[89]: 495

In [82]:
marketing.groupby('landing_page_id').count().show()

+--------------------+-----+
 landing_page_id|count|
+--------------------+-----+
a7982125ff7aa3b20...| 156|
9df372c20395ad99d...| 5|
2228a43ac0bc372e2...| 20|
f92b00f4abed4326f...| 1|
68f817c510c978e3a...| 11|
cb049134a163b197a...| 2|
7e11885cec8e2f849...| 1|
891b03aa737ba5655...| 3|
5c517d06e51b50ae5...| 1|
3aea860a87895eee6...| 1|
9496ccbb759ca86ff...| 1|
dfd5e75bc022dc8b7...| 1|
57d5edb1844a74612...| 9|
b21580ecd6fb023c6...| 1|
447cabeecc99dc245...| 3|
dabf7461e7b9fbcb0...| 1|
8345a83d419160e69...| 1|
5215a2f4c4cb554cb...| 1|
d1905a1a023dc6379...| 1|
8ca8a42a9a8f1c605...| 1|
+--------------------+-----+
only showing top 20 rows

In [83]:
landing = marketing.groupby('landing_page_id').count()
display(landing.orderBy('count', ascending=False))

landing_page_id,count
b76ef37428e6799c421989521c0e5077,912
22c29808c4f815213303f8933030604c,883
58326e62183c14b0c03085c33b9fdc44,495
88740e65d5d6b056e0cda098e1ea6313,445
ce1a65abd0973638f1c887a6efcfa82d,394
40dec9f3d5259a3d2dbcdab2114fae47,330
f017be4dbf86243af5c1ebed0cff36a2,310
e492ee5eaf1697716985cc6f33f9cd9b,291
a7982125ff7aa3b2054c6e44f9d28522,156
73f31a40697cc90a86c1273563ac230e,115


In [84]:
marketing.select('origin').distinct().count()

Out[92]: 11

In [85]:
marketing.groupby('origin').count().show()

+-----------------+-----+
 origin|count|
+-----------------+-----+
 NA| 60|
 unknown| 1099|
 display| 118|
 referral| 284|
 other| 150|
 email| 493|
 social| 1350|
other_publicities| 65|
 direct_traffic| 499|
 organic_search| 2296|
 paid_search| 1586|
+-----------------+-----+

In [86]:
origem = marketing.groupby('origin').count()
display(origem.orderBy('count', ascending=False))

origin,count
organic_search,2296
paid_search,1586
social,1350
unknown,1099
direct_traffic,499
email,493
referral,284
other,150
display,118
other_publicities,65


In [87]:
order_items.dtypes

Out[95]: [('order_id', 'string'),
 ('order_item_id', 'int'),
 ('product_id', 'string'),
 ('seller_id', 'string'),
 ('shipping_limit_date', 'timestamp'),
 ('price', 'double'),
 ('freight_value', 'double')]

In [88]:
display(order_items.describe())

summary,order_id,order_item_id,product_id,seller_id,price,freight_value
count,112650,112650,112650,112650,112650,112650
mean,null,1.1978339991122948,null,null,120.65373901464174,19.990319928982977
stddev,null,0.7051240313951721,null,null,183.6339280502595,15.806405412297098
min,00010242fe8c5a6d1ba2dd792cb16214,1,00066f42aeeb9f3007548bb9d3f33c38,0015a82c2db000af6aaaf3ae2ecb0532,0.85,0.0
max,fffe41c64501cc87c801fd61db3f6244,21,fffe9eeff12fcbd74a2f2b007dde0c58,ffff564a4f9085cd26170f4732393726,6735.0,409.68


In [89]:
from pyspark.sql.functions import isnull, when, count, col

aux = []
for c in order_items.columns:
  aux.append(count(when(isnull(c), c)).alias(c))

display(order_items.select(aux))

order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,0,0,0,0,0,0


In [90]:
order_items.select('order_id').distinct().count()

Out[98]: 98666

In [91]:
order_items.select('order_id').count()

Out[99]: 112650

In [92]:
order_items.groupby('order_id').count().show()

+--------------------+-----+
 order_id|count|
+--------------------+-----+
014405982914c2cde...| 2|
019886de8f385a39b...| 1|
01a6ad782455876aa...| 1|
01d907b3e209269e1...| 1|
028dc52e12ddda803...| 1|
036dd381dfb3ec75e...| 1|
03ebfa9712b7dbc70...| 1|
0420da8d50a378401...| 1|
05afef1c185862cab...| 2|
05bef443b85068505...| 1|
05d46826bd0fb605d...| 1|
06fec3f6e9e72edb6...| 1|
077700dcf4e3bb412...| 1|
078c2d9bb7e5905e8...| 2|
08791ec24b12af5af...| 1|
08b450b388221bfe5...| 1|
0957ed870116e596b...| 1|
09868b0eb512bfbbe...| 1|
099d60cb800db65a7...| 1|
09e90e3936db197d4...| 1|
+--------------------+-----+
only showing top 20 rows

In [93]:
order_id_rep = order_items.groupby('order_id').count()
display(order_id_rep.orderBy('count', ascending=False))

order_id,count
8272b63d03f5f79c56e9e4120aec44ef,21
1b15974a0141d54e36626dca3fdc731a,20
ab14fdcfbe524636d65ee38360e22ce8,20
428a2f660dc84138d969ccd69a0ab6d5,15
9ef13efd6949e4573a18964dd1bbe7f5,15
73c8ab38f07dc94389065f7eba4f297a,14
9bdc4d4c71aa1de4606060929dee888c,14
37ee401157a3a0b28c9c6d0ed8c3b24b,13
637617b3ffe9e2f7a2411243829226d0,12
2c2a19b5703863c908512d135aa6accc,12


In [94]:
order_items.select('order_item_id').distinct().count()

Out[102]: 21

In [95]:
order_items.groupby('order_item_id').count().show()

+-------------+-----+
order_item_id|count|
+-------------+-----+
 12| 13|
 1|98666|
 13| 8|
 6| 256|
 16| 3|
 3| 2287|
 20| 3|
 5| 460|
 19| 3|
 15| 5|
 9| 28|
 17| 3|
 4| 965|
 8| 36|
 7| 58|
 10| 25|
 21| 1|
 11| 17|
 14| 7|
 2| 9803|
+-------------+-----+
only showing top 20 rows

In [96]:
order_item_id_rep = order_items.groupby('order_item_id').count()
display(order_item_id_rep.orderBy('count', ascending=False))

order_item_id,count
1,98666
2,9803
3,2287
4,965
5,460
6,256
7,58
8,36
9,28
10,25


In [97]:
order_items.select('product_id').distinct().count()

Out[105]: 32951

In [98]:
order_items.groupby('product_id').count().show()

+--------------------+-----+
 product_id|count|
+--------------------+-----+
0b0172eb0fd18479d...| 16|
42a2bd596fda1baef...| 21|
460a66fcc404a3d73...| 6|
30360c8b0b2ac6918...| 3|
13b4ff901d43edec6...| 2|
290ada89b05e1dca2...| 2|
7724696de32f44179...| 1|
abe236a52dbc43e90...| 9|
e85cdca8790ea0026...| 10|
eb1549e153933dfee...| 1|
35bc6c77029697004...| 19|
878699846fa5ea02a...| 1|
2f9c2888168b8c2d8...| 6|
75f3ef6a5cb0f2d5a...| 6|
3f1a741cf55913844...| 20|
526e24e5c44ede53f...| 3|
be7f0912ba0112670...| 5|
18d87751433e4198e...| 3|
b23cfa104a0f29745...| 2|
5a57a59c44429be19...| 5|
+--------------------+-----+
only showing top 20 rows

In [99]:
product_id_rep = order_items.groupby('product_id').count()
display(product_id_rep.orderBy('count', ascending=False))

product_id,count
aca2eb7d00ea1a7b8ebd4e68314663af,527
99a4788cb24856965c36a24e339b6058,488
422879e10f46682990de24d770e7f83d,484
389d119b48cf3043d311335e499d9c6b,392
368c6c730842d78016ad823897a372db,388
53759a2ecddad2bb87a079a1f1519f73,373
d1c427060a0f73f6b889a5c7c61f2ac4,343
53b36df67ebb7c41585e8d54d6772e08,323
154e7e31ebfa092203795c972e5804a6,281
3dd2a17168ec895c781a9191c1e95ad7,274


In [100]:
order_items.select('seller_id').distinct().count()

Out[108]: 3095

In [101]:
order_items.select('seller_id').count()

Out[109]: 112650

In [102]:
order_items.groupby('seller_id').count().show()

+--------------------+-----+
 seller_id|count|
+--------------------+-----+
ff063b022a9a0aab9...| 96|
8e6cc767478edae94...| 114|
a49928bcdf77c55c6...| 106|
da7039f29f90ce5b4...| 18|
062ce95fa2ad4dfae...| 60|
2009a095de2a2a416...| 12|
0ea22c1cfbdc755f8...| 269|
6eeed17989b0ae47c...| 3|
e63e8bfa530fb1691...| 43|
4d600e08ecbe08258...| 8|
9803a40e82e45418a...| 18|
b3f19518fcec265b2...| 9|
ec8879960bd2221d5...| 36|
0b64bcdb0784abc13...| 4|
c522be04e020c1e7b...| 7|
9c068d10aca38e85c...| 17|
297d5eccd19fa9a83...| 8|
9b1050e85becf3ae9...| 2|
a3082f442524a1be4...| 4|
791cfcfe22fe4a771...| 2|
+--------------------+-----+
only showing top 20 rows

In [103]:
seller_id_rep = order_items.groupby('seller_id').count()
display(seller_id_rep.orderBy('count', ascending=False))

seller_id,count
6560211a19b47992c3666cc44a7e94c0,2033
4a3ca9315b744ce9f8e9374361493884,1987
1f50f920176fa81dab994f9023523100,1931
cc419e0650a3c5ba77189a1882b7556a,1775
da8622b14eb17ae2831f4ac5b9dab84a,1551
955fee9216a65b617aa5c0531780ce60,1499
1025f0e2d44d7041d6cf58b6550e0bfa,1428
7c67e1448b00f6e969d365cea6b010ab,1364
ea8482cd71df3c1969d7b9473ff13abc,1203
7a67c85e85bb2ce8582c35f2203ad736,1171


In [104]:
order_items.select('shipping_limit_date').distinct().count()

Out[112]: 93318

In [105]:
order_items.select('shipping_limit_date').count()

Out[113]: 112650

In [106]:
order_items.groupby('shipping_limit_date').count().show()

+-------------------+-----+
shipping_limit_date|count|
+-------------------+-----+
2017-10-26 00:15:00| 1|
2018-02-06 14:13:31| 1|
2017-05-11 22:35:11| 1|
2017-03-19 23:23:23| 1|
2017-03-29 20:02:41| 3|
2017-11-06 11:09:50| 1|
2017-08-31 02:24:28| 1|
2017-08-10 13:10:18| 1|
2017-12-08 16:30:46| 2|
2018-04-03 14:15:45| 1|
2017-09-14 02:55:29| 1|
2018-04-27 01:30:43| 2|
2017-11-08 09:35:30| 1|
2018-01-05 18:09:23| 1|
2017-05-29 09:02:41| 1|
2017-01-18 18:27:28| 1|
2017-04-28 13:30:12| 1|
2018-07-25 11:05:14| 2|
2018-08-02 21:15:18| 1|
2018-07-27 02:50:16| 2|
+-------------------+-----+
only showing top 20 rows

In [107]:
display(order_items.describe('price'))

summary,price
count,112650
mean,120.65373901464174
stddev,183.6339280502595
min,0.85
max,6735.0


In [108]:
order_items.approxQuantile("price", [0.5], 0.25)

Out[116]: [75.0]

In [109]:
display(order_items.select('price'))

price
58.9
239.9
199.0
12.99
199.9
21.9
19.9
810.0
145.95
53.99


In [110]:
display(order_items.select('price'))

price
58.9
239.9
199.0
12.99
199.9
21.9
19.9
810.0
145.95
53.99


In [111]:
price_rep = order_items.groupby('price').count()
display(price_rep.orderBy('count', ascending=False))

#R$59,90 é o preço que mais se repete

price,count
59.9,2481
69.9,1987
49.9,1953
89.9,1548
99.9,1432
39.9,1339
29.9,1318
79.9,1214
19.9,1201
29.99,1176


In [112]:
order_items.select('freight_value').distinct().count()

Out[120]: 6999

In [113]:
order_items.groupby('freight_value').count().show()

+-------------+-----+
freight_value|count|
+-------------+-----+
 14.9| 38|
 17.95| 194|
 19.98| 61|
 13.4| 37|
 8.51| 35|
 26.7| 13|
 23.04| 22|
 26.72| 10|
 9.13| 20|
 17.56| 19|
 74.5| 1|
 12.32| 24|
 10.65| 10|
 77.19| 2|
 40.53| 4|
 17.52| 22|
 15.5| 48|
 67.21| 2|
 49.8| 2|
 58.92| 4|
+-------------+-----+
only showing top 20 rows

In [114]:
display(order_items.describe('freight_value'))

summary,freight_value
count,112650
mean,19.990319928982977
stddev,15.806405412297098
min,0.0
max,409.68


In [115]:
order_items.approxQuantile("freight_value", [0.5], 0.25)

Out[123]: [16.24]

In [116]:
display(order_items.select('freight_value'))

freight_value
13.29
19.93
17.87
12.79
18.14
12.69
11.85
70.75
11.65
11.4


In [117]:
display(order_items.select('freight_value'))

freight_value
13.29
19.93
17.87
12.79
18.14
12.69
11.85
70.75
11.65
11.4


In [118]:
frete = order_items.groupby('freight_value').count()
display(frete.orderBy('count', ascending=False))

#R$15,10 é o frete mais comum.

freight_value,count
15.1,3707
7.78,2262
14.1,1875
11.85,1846
18.23,1575
7.39,1521
16.11,1152
15.23,1010
8.72,921
16.79,873


In [119]:
order_payments.dtypes

Out[127]: [('order_id', 'string'),
 ('payment_sequential', 'int'),
 ('payment_type', 'string'),
 ('payment_installments', 'int'),
 ('payment_value', 'double')]

In [120]:
display(order_payments.describe())

summary,order_id,payment_sequential,payment_type,payment_installments,payment_value
count,103886,103886,103886,103886,103886
mean,null,1.0926785129853878,null,2.853348863176944,154.10038041698792
stddev,null,0.7065837791949958,null,2.687050673856492,217.494063864724
min,00010242fe8c5a6d1ba2dd792cb16214,1,boleto,0,0.0
max,fffe41c64501cc87c801fd61db3f6244,29,voucher,24,13664.08


In [121]:
from pyspark.sql.functions import isnull, when, count, col

aux = []
for c in order_payments.columns:
  aux.append(count(when(isnull(c), c)).alias(c))

display(order_payments.select(aux))

order_id,payment_sequential,payment_type,payment_installments,payment_value
0,0,0,0,0


In [122]:
order_payments.select('order_id').distinct().count()

Out[130]: 99440

In [123]:
order_payments.select('order_id').count()

Out[131]: 103886

In [124]:
order_payments.groupby('order_id').count().show()

+--------------------+-----+
 order_id|count|
+--------------------+-----+
bb2d7e3141540afc2...| 1|
85be7c94bcd3f908f...| 1|
8ca5bdac5ebe8f2d6...| 9|
54066aeaaf3ac32e7...| 2|
5db54d41d5ebd6d76...| 1|
41537821ce113ccef...| 1|
beca5b5e9460824d8...| 1|
33f1e992ba3e439bf...| 1|
3fa59277573f0fe06...| 1|
e239d280236cdd3c4...| 1|
81d715926d69a15e7...| 1|
3e654c7f3c4a852f6...| 1|
66b9c991ee308f934...| 1|
c1101a911387fc42d...| 1|
281f470497aa1a0af...| 1|
cae8bf54a0ecd7937...| 1|
8afeb7bc3891eac7f...| 1|
b0200f320613faea2...| 1|
2d58f989c5e934455...| 1|
0d4a43cf7b7994e0a...| 1|
+--------------------+-----+
only showing top 20 rows

In [125]:
order_payments.select('payment_sequential').distinct().count()

Out[133]: 29

In [126]:
order_payments.groupby('payment_sequential').count().show()

+------------------+-----+
payment_sequential|count|
+------------------+-----+
 28| 1|
 27| 1|
 26| 2|
 12| 21|
 22| 3|
 1|99360|
 13| 13|
 6| 118|
 16| 6|
 3| 581|
 20| 4|
 5| 170|
 19| 6|
 15| 8|
 17| 6|
 9| 43|
 4| 278|
 8| 54|
 23| 2|
 7| 82|
+------------------+-----+
only showing top 20 rows

In [127]:
order_payments.select('payment_type').distinct().count()

Out[135]: 5

In [128]:
order_payments.groupby('payment_type').count().show()

+------------+-----+
payment_type|count|
+------------+-----+
 boleto|19784|
 not_defined| 3|
 credit_card|76795|
 voucher| 5775|
 debit_card| 1529|
+------------+-----+

In [129]:
metodo_pagamento = order_payments.groupby('payment_type').count()
display(metodo_pagamento.orderBy('count', ascending=False))

payment_type,count
credit_card,76795
boleto,19784
voucher,5775
debit_card,1529
not_defined,3


In [130]:
order_payments.select('payment_installments').distinct().count()

Out[138]: 24

In [131]:
order_payments.groupby('payment_installments').count().show()

+--------------------+-----+
payment_installments|count|
+--------------------+-----+
 12| 133|
 22| 1|
 1|52546|
 13| 16|
 6| 3920|
 16| 5|
 3|10461|
 20| 17|
 5| 5239|
 15| 74|
 9| 644|
 17| 8|
 4| 7098|
 8| 4268|
 23| 1|
 7| 1626|
 10| 5328|
 24| 18|
 21| 3|
 11| 23|
+--------------------+-----+
only showing top 20 rows

In [132]:
parcelas = order_payments.groupby('payment_installments').count()
display(parcelas.orderBy('count', ascending=False))

#O pagamento a vista é o mais escolhido

payment_installments,count
1,52546
2,12413
3,10461
4,7098
10,5328
5,5239
8,4268
6,3920
7,1626
9,644


In [133]:
order_payments.select('payment_value').distinct().count()

Out[141]: 29077

In [134]:
order_payments.groupby('payment_value').count().show()

+-------------+-----+
payment_value|count|
+-------------+-----+
 41.42| 10|
 69.8| 2|
 348.74| 1|
 330.4| 1|
 40.53| 9|
 134.49| 10|
 2276.1| 1|
 211.6| 7|
 165.39| 2|
 77.19| 9|
 244.8| 2|
 58.92| 3|
 82.51| 8|
 1338.41| 3|
 70.07| 10|
 50.74| 4|
 126.86| 8|
 17.52| 2|
 168.5| 4|
 53.82| 8|
+-------------+-----+
only showing top 20 rows

In [135]:
display(order_payments.describe('payment_value'))

summary,payment_value
count,103886
mean,154.10038041698792
stddev,217.494063864724
min,0.0
max,13664.08


In [136]:
order_payments.approxQuantile("payment_value", [0.5], 0.25)

Out[144]: [57.12]

In [137]:
display(order_items.select('freight_value'))

freight_value
13.29
19.93
17.87
12.79
18.14
12.69
11.85
70.75
11.65
11.4


In [138]:
display(order_items.select('freight_value'))

freight_value
13.29
19.93
17.87
12.79
18.14
12.69
11.85
70.75
11.65
11.4


In [139]:
valor_pagamento = order_payments.groupby('payment_value').count()
display(valor_pagamento.orderBy('count', ascending=False))

#R$50,00 é o valor mais comum nesta base.

payment_value,count
50.0,324
20.0,274
100.0,255
77.57,250
35.0,165
73.34,160
30.0,133
116.94,131
56.78,122
65.0,116


In [140]:
order_review.dtypes

Out[148]: [('review_id', 'string'),
 ('order_id', 'string'),
 ('review_score', 'string'),
 ('review_comment_title', 'string'),
 ('review_comment_message', 'string'),
 ('review_creation_date', 'string'),
 ('review_answer_timestamp', 'string')]

In [141]:
display(order_review.describe())

summary,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
count,105188,102859,102692,12176,41868,96025,96002
mean,4.5,0.0,4.071667849964501,3.1554493965252365E10,1.1111111111111111E198,null,null
stddev,0.7071067811865476,0.0,1.386648877434681,5.616554832455847E11,Infinity,null,null
min,"""",,"""",,,FOI A MINHA PRIMEIRA COMPRA E NÃO RECOMENDO,POIS NÃO CUMPREM PARA DE ENTREGA ESTABELECIDO.
max,"🤙🏼👏🏼👏🏼""",visando sempre o melhor para os clientes!,"seria mais coerente.""",🔟,😡😡😡😡😡👎👎👎👎👎,"veio bem embalada oque indica que foi embalada ja quebrada.triste""",70 + R$15


In [142]:
from pyspark.sql.functions import isnull, when, count, col

aux = []
for c in order_review.columns:
  aux.append(count(when(isnull(c), c)).alias(c))

display(order_review.select(aux))

review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
1,2330,2497,93013,63321,9164,9187


In [143]:
order_review = order_review.fillna('NA', subset=['review_id'])
order_review = order_review.fillna('NA', subset=['order_id'])
order_review = order_review.fillna('NA', subset=['review_comment_title'])
order_review = order_review.fillna('NA', subset=['review_comment_message'])
order_review = order_review.fillna('NA', subset=['review_creation_date'])
order_review = order_review.fillna('NA', subset=['review_answer_timestamp'])

In [144]:
order_review = order_review.dropna()
order_review.count()

Out[152]: 102692

In [145]:
from pyspark.sql.functions import isnull, when, count, col

aux = []
for c in order_review.columns:
  aux.append(count(when(isnull(c), c)).alias(c))

display(order_review.select(aux))

review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,0,0,0,0,0,0


In [146]:
display(order_review.describe())

summary,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
count,102692,102692,102692,102692,102692,102692,102692
mean,4.0,0.0,4.071667849964501,3.1554493965252365E10,1.1111111111111111E198,null,null
stddev,NaN,NaN,1.386648877434681,5.616554832455847E11,Infinity,null,null
min,"""","""","""",,,FOI A MINHA PRIMEIRA COMPRA E NÃO RECOMENDO,POIS NÃO CUMPREM PARA DE ENTREGA ESTABELECIDO.
max,"🤙🏼👏🏼👏🏼""","são bem feito.""","seria mais coerente.""",🔟,😡😡😡😡😡👎👎👎👎👎,"veio bem embalada oque indica que foi embalada ja quebrada.triste""",NA


In [147]:
order_review.select('review_id').distinct().count()

Out[155]: 101541

In [148]:
order_review.select('review_id').count()

Out[156]: 102692

In [149]:
order_review.groupby('review_id').count().show()

+--------------------+-----+
 review_id|count|
+--------------------+-----+
b189da69da7ab9594...| 1|
cea53cba54c5c3a5d...| 1|
a835f8e62dd58cf9c...| 1|
dafd8ae6a06afff04...| 1|
b200a329d942243a5...| 1|
974929cd0aae95525...| 1|
9a10c75418c797395...| 1|
967749bfad96e5603...| 1|
c83adde7bfa20debf...| 1|
41f60ee2c98b4ab84...| 1|
ca2afdf36803f89a7...| 1|
050dc4d6c3121ff33...| 1|
d2aa0192c8dffa354...| 1|
63920f731a16cb9f2...| 1|
88a77df7b5f139b25...| 1|
dde5b1627aac00c45...| 1|
72cbb1369ba5acfee...| 1|
108f9566f87b526f6...| 1|
b9c0332e487c78fa3...| 1|
5a80c6e2e285918e7...| 1|
+--------------------+-----+
only showing top 20 rows

In [150]:
review = order_review.groupby('review_id').count()
display(review.orderBy('count', ascending=False))

review_id,count
"""",29
"Recomendo.""",21
"Recomendo""",21
"Obrigada""",18
"Recomendo!""",14
"Recomendo """,14
"Parabéns """,9
"Obrigado""",9
"Recomendo. """,8
"Obrigada!""",8


In [151]:
review = order_review.groupby('review_id').count()
display(review.orderBy('count', ascending=False))

review_id,count
"""",29
"Recomendo.""",21
"Recomendo""",21
"Obrigada""",18
"Recomendo!""",14
"Recomendo """,14
"Parabéns """,9
"Obrigado""",9
"Recomendo. """,8
"Obrigada!""",8


In [152]:
order_review.select('order_id').distinct().count()

Out[160]: 100402

In [153]:
order_review.groupby('order_id').count().show()

+--------------------+-----+
 order_id|count|
+--------------------+-----+
e239d280236cdd3c4...| 1|
d451da9b109e1786f...| 1|
0de5dbc9f32267616...| 1|
251f0a3981c4a8cb8...| 1|
f63a31c3349b87273...| 2|
800c2d74bc71b0e27...| 1|
2634d2363b155e0d1...| 1|
1c7b6d63329bdc850...| 1|
5347a7f07b3387d2d...| 1|
0bcc4cc85c24f23e2...| 1|
2c88b5879d666444b...| 1|
48e6590a35c4212b1...| 1|
a9f77c0325604197d...| 1|
fcb5deccd553af305...| 1|
0d4a43cf7b7994e0a...| 1|
a684a880595e155b0...| 1|
d43547ebdfd7f5f70...| 1|
fc79d61f2a305370f...| 1|
478e0f644b5e774c5...| 1|
c349154248a3236a0...| 1|
+--------------------+-----+
only showing top 20 rows

In [154]:
order_review.select('review_score').distinct().count()

Out[162]: 2567

In [155]:
order_review.select('review_score').count()

Out[163]: 102692

In [156]:
order_review.groupby('review_score').count().show()

+--------------------+-----+
 review_score|count|
+--------------------+-----+
 2018-01-27 20:03:09| 1|
 2017-07-09 00:33:40| 1|
 2017-10-21 01:12:49| 1|
 2018-04-16 19:34:18| 1|
 2017-03-28 22:38:46| 1|
 2018-05-04 02:18:31| 1|
 2017-12-14 20:09:47| 1|
 2018-01-30 16:02:47| 1|
 2018-01-04 17:32:51| 1|
 alias comprei ju...| 1|
 2018-06-17 20:29:17| 1|
 2017-02-10 03:19:21| 1|
 2018-05-04 22:37:24| 1|
 não acondicionad...| 1|
 2018-02-22 22:10:53| 1|
 2018-03-04 18:14:09| 1|
 2018-04-01 21:21:17| 1|
 2017-12-18 12:20:25| 1|
 2017-09-03 00:00:00| 1|
 2017-07-21 00:11:57| 1|
+--------------------+-----+
only showing top 20 rows

In [157]:
pontuacao = order_review.groupby('review_score').count()
display(pontuacao.orderBy('count', ascending=False))

# É necessário um tratamento dessa variável

review_score,count
5,57420
4,19200
1,11858
3,8287
2,3235
2018-08-09 00:00:00,5
2018-05-19 00:00:00,5
2018-07-27 00:00:00,4
2017-12-29 00:00:00,4
2018-01-04 00:00:00,4


In [158]:
pontuacao = order_review.groupby('review_score').count()
display(pontuacao.orderBy('count', ascending=False))

review_score,count
5,57420
4,19200
1,11858
3,8287
2,3235
2018-08-09 00:00:00,5
2018-05-19 00:00:00,5
2018-07-27 00:00:00,4
2017-12-29 00:00:00,4
2018-03-17 00:00:00,4


In [159]:
order_review.select('review_comment_title').distinct().count()

Out[167]: 5044

In [160]:
order_review.select('review_comment_title').count()

Out[168]: 102692

In [161]:
order_review.groupby('review_comment_title').count().show()

+--------------------+-----+
review_comment_title|count|
+--------------------+-----+
 2018-05-22 13:13:15| 1|
 2018-02-21 00:14:15| 1|
 Cadeira| 3|
ENTREGA DENTRO DO...| 1|
 Decepção!| 2|
 Entrega ok| 5|
 Primeira v| 1|
 Ótima !!| 1|
 ...| 1|
 Adesivos amassados| 1|
 aspecto do produto| 1|
 Surpresa!| 1|
 Produto não é bom!| 1|
 lindo | 1|
Grelha pra churra...| 1|
 super recomendo-10| 1|
 Produto Incompleto| 2|
 Otimo!!| 1|
 SUPER RECOPMENDADO| 1|
 Horrível | 5|
+--------------------+-----+
only showing top 20 rows

In [162]:
titulo = order_review.groupby('review_comment_title').count()
display(titulo.orderBy('count', ascending=False))

review_comment_title,count
NA,90517
Recomendo,426
recomendo,346
Bom,294
super recomendo,271
Excelente,248
Muito bom,247
Ótimo,240
Super recomendo,218
Ótimo,207


In [163]:
order_review.select('review_comment_message').distinct().count()

Out[171]: 36878

In [164]:
order_review.groupby('review_comment_message').count().show()

+----------------------+-----+
review_comment_message|count|
+----------------------+-----+
 Toalha do maior c...| 1|
 Tudo conforme e e...| 1|
 Boa tarde meu pro...| 1|
 Gosto muito de co...| 1|
 Amei a tesoura co...| 1|
 valeu a pena entr...| 1|
 Tudo ótimo 
 1|
 não recomendo poi...| 1|
 produto entregue ...| 1|
 Efetuei a compra ...| 1|
 Produto diferente...| 1|
 Recebi o produto ...| 1|
 A data limite par...| 1|
 ...| 11|
 Entrega antes do ...| 1|
 Envio muito rápid...| 1|
 Como se trata de ...| 1|
 Recebi o produto ...| 1|
 Fiz a compra de d...| 1|
 Melhor preço,prod...| 1|
+----------------------+-----+
only showing top 20 rows

In [165]:
mensagens = order_review.groupby('review_comment_message').count()
display(mensagens.orderBy('count', ascending=False))

review_comment_message,count
NA,60825
Muito bom,230
Bom,189
muito bom,122
bom,107
Recomendo,100
Otimo,99
otimo,96
Ótimo,77
Ótimo,74


In [166]:
order_review.select('review_creation_date').distinct().count()

Out[174]: 726

In [167]:
order_review.groupby('review_creation_date').count().show()

+--------------------+-----+
review_creation_date|count|
+--------------------+-----+
 2017-05-08 00:00:00| 20|
 2017-09-03 00:00:00| 47|
 então tudo ótimo...| 1|
 2017-04-21 00:00:00| 138|
 2017-10-27 00:00:00| 199|
 2018-05-16 00:00:00| 361|
 2017-05-18 00:00:00| 163|
 2017-04-18 00:00:00| 113|
 2017-02-22 00:00:00| 89|
 nem vou me estre...| 1|
 2018-03-28 00:00:00| 350|
 2017-08-26 00:00:00| 203|
 2018-01-03 00:00:00| 35|
 porém veio somen...| 1|
 2017-05-12 00:00:00| 140|
veio bem embalada...| 1|
 2016-12-04 00:00:00| 4|
 com isso tive qu...| 1|
 2017-12-19 00:00:00| 448|
 2017-05-13 00:00:00| 156|
+--------------------+-----+
only showing top 20 rows

In [168]:
data = order_review.groupby('review_creation_date').count()
display(data.orderBy('count', ascending=False))

review_creation_date,count
NA,6667
2017-12-19 00:00:00,448
2018-08-28 00:00:00,435
2017-12-20 00:00:00,430
2018-05-19 00:00:00,417
2018-08-14 00:00:00,408
2018-03-29 00:00:00,408
2018-05-22 00:00:00,407
2018-05-15 00:00:00,402
2018-05-04 00:00:00,396


In [169]:
order_review.select('review_answer_timestamp').distinct().count()

Out[177]: 95073

In [170]:
order_review.groupby('review_answer_timestamp').count().show()

+-----------------------+-----+
review_answer_timestamp|count|
+-----------------------+-----+
 2018-05-25 03:45:28| 1|
 2018-05-10 17:19:17| 1|
 2018-02-06 11:51:18| 1|
 2018-08-22 18:33:02| 1|
 2017-11-06 22:29:34| 1|
 2018-02-26 13:59:17| 1|
 2017-11-05 10:43:54| 1|
 2018-04-21 03:31:59| 1|
 2018-01-26 22:29:43| 1|
 2017-05-22 12:05:04| 1|
 2018-08-11 21:20:30| 1|
 2018-08-02 12:20:27| 1|
 2018-05-21 02:29:27| 1|
 2017-12-24 20:07:54| 1|
 2017-05-01 12:44:11| 1|
 2017-12-07 09:43:36| 1|
 2017-07-23 21:36:45| 1|
 2018-02-09 10:19:32| 1|
 2017-11-03 07:01:55| 1|
 2018-08-22 14:29:05| 1|
+-----------------------+-----+
only showing top 20 rows

In [171]:
data_resp = order_review.groupby('review_answer_timestamp').count()
display(data_resp.orderBy('count', ascending=False))

review_answer_timestamp,count
NA,6690
2017-06-15 23:21:05,4
2018-06-17 23:48:04,3
2018-07-31 23:11:57,3
2018-03-30 00:29:09,3
2018-08-31 22:29:09,3
2017-08-12 14:35:03,3
2017-08-30 11:18:29,3
2017-09-05 15:27:17,3
2017-11-02 00:11:06,3


In [172]:
orders.dtypes

Out[180]: [('order_id', 'string'),
 ('customer_id', 'string'),
 ('order_status', 'string'),
 ('order_purchase_timestamp', 'timestamp'),
 ('order_approved_at', 'timestamp'),
 ('order_delivered_carrier_date', 'timestamp'),
 ('order_delivered_customer_date', 'timestamp'),
 ('order_estimated_delivery_date', 'timestamp')]

In [173]:
display(orders.describe())

summary,order_id,customer_id,order_status
count,99441,99441,99441
mean,null,null,null
stddev,null,null,null
min,00010242fe8c5a6d1ba2dd792cb16214,00012a2ce6f8dcda20d059ce98491703,approved
max,fffe41c64501cc87c801fd61db3f6244,ffffe8b65bbe3087b653a978c870db99,unavailable


In [174]:
from pyspark.sql.functions import isnull, when, count, col

aux = []
for c in orders.columns:
  aux.append(count(when(isnull(c), c)).alias(c))

display(orders.select(aux))

order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,0,0,0,160,1783,2965,0


In [175]:
orders = orders.fillna('NA', subset=['order_approved_at'])
orders = orders.fillna('NA', subset=['order_delivered_carrier_date'])
orders = orders.fillna('NA', subset=['order_delivered_customer_date'])

In [176]:
orders = orders.dropna()
orders.count()

In [177]:
from pyspark.sql.functions import isnull, when, count, col

aux = []
for c in orders.columns:
  aux.append(count(when(isnull(c), c)).alias(c))

display(orders.select(aux))

In [178]:
display(orders.describe())

In [179]:
orders.select('order_id').distinct().count()

In [180]:
orders.select('order_id').count()

In [181]:
orders.groupby('order_id').count().show()

In [182]:
orders.select('customer_id').distinct().count()

In [183]:
orders.select('customer_id').count()

In [184]:
orders.groupby('customer_id').count().show()

In [185]:
orders.select('order_status').distinct().count()

In [186]:
orders.groupby('order_status').count().show()

In [187]:
orders.select('order_purchase_timestamp').distinct().count()

In [188]:
orders.groupby('order_purchase_timestamp').count().show()

In [189]:
timestamp_compra = orders.groupby('order_purchase_timestamp').count()
display(timestamp_compra.orderBy('count', ascending=False))

In [190]:
orders.select('order_approved_at').distinct().count()

In [191]:
orders.groupby('order_approved_at').count().show()

In [192]:
timestamp_aprovacao = orders.groupby('order_approved_at').count()
display(timestamp_aprovacao.orderBy('count', ascending=False))

In [193]:
orders.select('order_delivered_carrier_date').distinct().count()

In [194]:
orders.groupby('order_delivered_carrier_date').count().show()

In [195]:
timestamp_transportadora = orders.groupby('order_delivered_carrier_date').count()
display(timestamp_transportadora.orderBy('count', ascending=False))

#TEM MUITO NULO!!!

In [196]:
orders.select('order_delivered_customer_date').distinct().count()

In [197]:
orders.groupby('order_delivered_customer_date').count().show()

In [198]:
timestamp_cliente = orders.groupby('order_delivered_customer_date').count()
display(timestamp_cliente.orderBy('count', ascending=False))

#Tem muito nulo!

In [199]:
orders.select('order_estimated_delivery_date').distinct().count()

In [200]:
orders.groupby('order_estimated_delivery_date').count().show()

In [201]:
timestamp_estimado = orders.groupby('order_estimated_delivery_date').count()
display(timestamp_estimado.orderBy('count', ascending=False))

In [202]:
products.dtypes

In [203]:
display(products.describe())

In [204]:
from pyspark.sql.functions import isnull, when, count, col

aux = []
for c in products.columns:
  aux.append(count(when(isnull(c), c)).alias(c))

display(products.select(aux))

In [205]:
products = products.fillna('NA', subset=['product_category_name'])

In [206]:
products = products.dropna()
products.count()

In [207]:
lenght = products[['product_name_lenght']].agg({'product_name_lenght': 'mean'}).collect()[0][0]
products = products.fillna(lenght, subset=['product_name_lenght'])

descrip = products[['product_description_lenght']].agg({'product_description_lenght': 'mean'}).collect()[0][0]
products = products.fillna(descrip, subset=['product_description_lenght'])

photo = products[['product_photos_qty']].agg({'product_photos_qty': 'mean'}).collect()[0][0]
products = products.fillna(photo, subset=['product_photos_qty'])

weight = products[['product_weight_g']].agg({'product_weight_g': 'mean'}).collect()[0][0]
products = products.fillna(weight, subset=['product_weight_g'])

lenght_cm = products[['product_length_cm']].agg({'product_length_cm': 'mean'}).collect()[0][0]
products = products.fillna(lenght_cm, subset=['product_length_cm'])

height = products[['product_height_cm']].agg({'product_height_cm': 'mean'}).collect()[0][0]
products = products.fillna(height, subset=['product_height_cm'])

width = products[['product_width_cm']].agg({'product_width_cm': 'mean'}).collect()[0][0]
products = products.fillna(width, subset=['product_width_cm'])

In [208]:
from pyspark.sql.functions import isnull, when, count, col

aux = []
for c in products.columns:
  aux.append(count(when(isnull(c), c)).alias(c))

display(products.select(aux))

In [209]:
display(products.describe())

In [210]:
products.select('product_id').distinct().count()

In [211]:
products.select('product_id').count()

In [212]:
products.groupby('product_id').count().show()

In [213]:
products.select('product_category_name').distinct().count()

In [214]:
products.groupby('product_category_name').count().show()

In [215]:
categoria_produtos = products.groupby('product_category_name').count()
display(categoria_produtos.orderBy('count', ascending=False))

#TEM MUITO NULO!!!

In [216]:
display(categoria_produtos.orderBy('count', ascending=False).head(10))

In [217]:
products.select('product_name_lenght').distinct().count()

In [218]:
products.groupby('product_name_lenght').count().show()

In [219]:
products.select('product_description_lenght').distinct().count()

In [220]:
products.groupby('product_description_lenght').count().show()

In [221]:
display(products.describe('product_description_lenght'))

In [222]:
products.approxQuantile("product_description_lenght", [0.5], 0.25)

In [223]:
display(products.select('product_description_lenght'))

In [224]:
display(products.select('product_description_lenght'))

In [225]:
products.select('product_photos_qty').distinct().count()

In [226]:
products.groupby('product_photos_qty').count().show()

In [227]:
display(products.describe('product_photos_qty'))

In [228]:
products.approxQuantile("product_photos_qty", [0.5], 0.25)

In [229]:
display(products.select('product_photos_qty'))

In [230]:
display(products.select('product_photos_qty'))

In [231]:
products.select('product_weight_g').distinct().count()

In [232]:
display(products.describe('product_weight_g'))

In [233]:
products.groupby('product_weight_g').count().show()

In [234]:
products.approxQuantile("product_weight_g", [0.5], 0.25)

In [235]:
display(products.select('product_weight_g'))

In [236]:
display(products.select('product_weight_g'))

In [237]:
products.select('product_length_cm').distinct().count()

In [238]:
display(products.describe('product_length_cm'))

In [239]:
products.groupby('product_length_cm').count().show()

In [240]:
products.approxQuantile("product_length_cm", [0.5], 0.25)

In [241]:
display(products.select('product_length_cm'))

In [242]:
display(products.select('product_length_cm'))

In [243]:
products.select('product_height_cm').distinct().count()

In [244]:
display(products.describe('product_height_cm'))

In [245]:
products.groupby('product_height_cm').count().show()

In [246]:
products.approxQuantile("product_height_cm", [0.5], 0.25)

In [247]:
display(products.select('product_height_cm'))

In [248]:
display(products.select('product_height_cm'))

In [249]:
products.select('product_width_cm').distinct().count()

In [250]:
display(products.describe('product_width_cm'))

#A média de largura dos produtos é 23,2 cm

In [251]:
products.groupby('product_width_cm').count().show()

In [252]:
products.approxQuantile("product_width_cm", [0.5], 0.25)

In [253]:
display(products.select('product_width_cm'))

In [254]:
display(products.select('product_width_cm'))

In [255]:
sellers.dtypes

In [256]:
display(sellers.describe())

In [257]:
from pyspark.sql.functions import isnull, when, count, col

aux = []
for c in sellers.columns:
  aux.append(count(when(isnull(c), c)).alias(c))

display(sellers.select(aux))

In [258]:
sellers.select('seller_id').distinct().count()

In [259]:
sellers.select('seller_id').count()

In [260]:
sellers.groupby('seller_id').count().show()

In [261]:
sellers.select('seller_zip_code_prefix').distinct().count()

In [262]:
sellers.select('seller_zip_code_prefix').count()

In [263]:
sellers.groupby('seller_zip_code_prefix').count().show()

In [264]:
CEP = sellers.groupby('seller_zip_code_prefix').count()
display(CEP.orderBy('count', ascending=False))

In [265]:
sellers.select('seller_city').distinct().count()

In [266]:
sellers.select('seller_city').count()

In [267]:
sellers.groupby('seller_city').count().show()

In [268]:
cidade_vendedores = sellers.groupby('seller_city').count()
display(cidade_vendedores.orderBy('count', ascending=False))

In [269]:
sellers.select('seller_state').distinct().count()

In [270]:
sellers.select('seller_state').count()

In [271]:
sellers.groupby('seller_state').count().show()

In [272]:
estado_vendedores = sellers.groupby('seller_state').count()
display(estado_vendedores.orderBy('count', ascending=False))

In [273]:
product_category.dtypes

In [274]:
display(product_category.describe())

In [275]:
from pyspark.sql.functions import isnull, when, count, col

aux = []
for c in product_category.columns:
  aux.append(count(when(isnull(c), c)).alias(c))

display(product_category.select(aux))

In [276]:
product_category.select('product_category_name').distinct().count()

In [277]:
product_category.select('product_category_name').count()

In [278]:
product_category.groupby('product_category_name').count().show()

In [279]:
product_category.select('product_category_name_english').distinct().count()

In [280]:
product_category.groupby('product_category_name_english').count().show()

In [281]:
geolocation.dtypes

In [282]:
display(geolocation.describe())

In [283]:
from pyspark.sql.functions import isnull, when, count, col

aux = []
for c in geolocation.columns:
  aux.append(count(when(isnull(c), c)).alias(c))

display(geolocation.select(aux))

In [284]:
geolocation.select('geolocation_zip_code_prefix').distinct().count()

In [285]:
geolocation.select('geolocation_zip_code_prefix').count()

In [286]:
geolocation.groupby('geolocation_zip_code_prefix').count().show()

In [287]:
geolocation.select('geolocation_lat').distinct().count()

In [288]:
geolocation.groupby('geolocation_lat').count().show()

In [289]:
geolocation.select('geolocation_lng').distinct().count()

In [290]:
geolocation.groupby('geolocation_lng').count().show()

In [291]:
geolocation.select('geolocation_city').distinct().count()

In [292]:
geolocation.groupby('geolocation_city').count().show()

In [293]:
cidade_geo = geolocation.groupby('geolocation_city').count()
display(cidade_geo.orderBy('count', ascending=False))

In [294]:
geolocation.select('geolocation_state').distinct().count()

In [295]:
geolocation.groupby('geolocation_state').count().show()

In [296]:
estado_geo = geolocation.groupby('geolocation_state').count()
display(estado_geo.orderBy('count', ascending=False))